In [1]:
pip install  mediapipe

     ---------------------------------------- 0.0/162.1 kB ? eta -:--:--
     ------------------------------------ - 153.6/162.1 kB 4.6 MB/s eta 0:00:01
     -------------------------------------- 162.1/162.1 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/199.7 kB ? eta -:--:--
     ------------------------------------- 199.7/199.7 kB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.5 MB ? eta -:--:--
    --------------------------------------- 1.1/50.5 MB 35.5 MB/s eta 0:00:02
   --- ------------------------------------ 4.8/50.5 MB 60.7 MB/s eta 0:00:01
   ------ --------------------------------- 8.7/50.5 MB 79.1 MB/s eta 0:00:01
   --------- ------------------------------ 11.4/50.5 MB 81.8 MB/s eta 0:00:01
   ------------ --------------------------- 16.1/50.5 MB 93.9 MB/s eta 0:00:01
   ---------------- ----------------------- 20.7/50.5 MB 108.8 MB/s eta 0:00:01
   ------------------- -------------------- 25.0/50.5 MB 108.8 MB/s eta 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.60.0 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
cap  = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
         # Recolor to RGB
        img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        #make detection
        results = pose.process(img)
        # Recolor to BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        #render detections
        mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                 )
        cv2.imshow('Mediapipe feed',img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

[x: 0.46100226
y: 0.25716305
z: -0.79581565
visibility: 0.9993954
, x: 0.50008476
y: 0.1888677
z: -0.7577481
visibility: 0.9992329
, x: 0.52460283
y: 0.18994556
z: -0.75774753
visibility: 0.9990025
, x: 0.5458636
y: 0.19405304
z: -0.7583841
visibility: 0.99937785
, x: 0.42479825
y: 0.19427143
z: -0.7431932
visibility: 0.99911124
, x: 0.40685958
y: 0.1971005
z: -0.7427475
visibility: 0.9987312
, x: 0.3923928
y: 0.20040464
z: -0.7431338
visibility: 0.999129
, x: 0.5938138
y: 0.23716116
z: -0.47689372
visibility: 0.99942064
, x: 0.38608152
y: 0.24866281
z: -0.39645234
visibility: 0.999033
, x: 0.5160049
y: 0.336789
z: -0.6927759
visibility: 0.99972135
, x: 0.43162006
y: 0.3492695
z: -0.66940445
visibility: 0.999749
, x: 0.7497497
y: 0.5864782
z: -0.42103052
visibility: 0.9977169
, x: 0.2971103
y: 0.637134
z: -0.19401887
visibility: 0.99733645
, x: 0.95901364
y: 0.9667208
z: -0.4621821
visibility: 0.7846926
, x: 0.022439063
y: 0.94052005
z: -0.20653892
visibility: 0.7803361
, x: 1.0286679


In [16]:
mp_drawing.DrawingSpec??

In [15]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.5598392
y: 0.5660143
z: -0.34072325
visibility: 0.9958237

In [4]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.8216692
y: 0.8845085
z: -0.3334301
visibility: 0.5092689

In [5]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 1.0466357
y: 1.156835
z: -0.86723006
visibility: 0.41000956

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #mid
    c = np.array(c) #end
    
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle >180.0:
        angle = 360-angle
    return angle

In [5]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [6]:
shoulder,elbow,wrist

([0.7704123258590698, 0.5842946171760559],
 [0.9669588804244995, 0.8614467978477478],
 [1.2105584144592285, 1.1399117708206177])

In [7]:
calculate_angle(shoulder,elbow,wrist)

174.16367668731007

In [8]:
cap  = cv2.VideoCapture(0)
# Curl counter
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
         # Recolor to RGB
        img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        #make detection
        results = pose.process(img)
        # Recolor to BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #Calculate angle
            angle = calculate_angle(shoulder,elbow,wrist)
            
            #Visualize
            cv2.putText(img,str(angle),tuple(np.multiply(elbow,[640,480]).astype(int)),
                                              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
            #Curl Counter logic
            if angle>160:
                stage = "down"
            if angle<30 and stage=='down':
                stage = 'up'
                counter += 1
                #print(counter)
        except:
            pass
        #Render curl counter
        #setup status box
        cv2.rectangle(img,(0,0),(225,73),(245,117,16),-1)
        #rep data
        cv2.putText(img,'REPS',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(img,str(counter),(10,60),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        
        #stage data
        cv2.putText(img,'STAGE',(65,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(img,stage,(60,60),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        
        #render detections
        mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                 )
        cv2.imshow('Mediapipe feed',img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()